In [1]:
import sys
import os


sys.path.append(os.path.abspath(".."))


In [2]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor

import matplotlib.pyplot as plt
import os
from datetime import datetime

# Import your environment
from env.stardew_mine_env import StardewMineEnv

/opt/miniconda3/envs/cs178/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [3]:
def make_env():
    env = StardewMineEnv(size=10, max_floor=10, max_energy=100, local_view_size=5)
    env = Monitor(env)  # Logs episode rewards
    return env

env = DummyVecEnv([make_env])


In [4]:
import os
from datetime import datetime


base_dir = os.getcwd()
parent_dir = os.path.dirname(base_dir)

models_dir = os.path.join(parent_dir, "models")
os.makedirs(models_dir, exist_ok=True) 

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_filename = f"ppo_miningbot_{timestamp}.zip"
save_path = os.path.join(models_dir, model_filename)

print("Full save path for model:", save_path)



Full save path for model: /Users/mingkunliu/Downloads/175/Strawdew_Valley_Mining_Bot/models/ppo_miningbot_20251130_233137.zip


In [ ]:
run_name = f"stardew_ppo_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

model = PPO(
    policy="MultiInputPolicy",
    env=env,
    verbose=1,
    learning_rate=3e-4,
    batch_size=64,
    n_steps=2048,
    gamma=0.99,
    tensorboard_log=f"logs/{run_name}"
)



NameError: name 'run_name' is not defined

In [9]:
TIMESTEPS = 200_000  # adjust for your experiment size

model.learn(total_timesteps=TIMESTEPS)
model.save(f"{save_path}/ppo_miningbot")
print("Training complete!")


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
model = PPO.load(f"{save_path}/ppo_miningbot")

test_env = StardewMineEnv()
obs, _ = test_env.reset()

total_reward = 0
done = False

while not done:
    action, _ = model.predict(obs)
    obs, reward, done, truncated, info = test_env.step(action)
    total_reward += reward

print("Test episode reward:", total_reward)

In [ ]:
import pandas as pd

monitor_file = [f for f in os.listdir(".") if f.endswith(".monitor.csv")]
monitor_file

In [ ]:
df = pd.read_csv(monitor_file[0], skiprows=1)

plt.plot(df["r"])  # r = reward
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.title("PPO Training Reward Curve")
plt.show()
